By Rithik Kumaran K

Text Summarization, Paraphrasing & Similarity Analyzer

This project is an end-to-end NLP pipeline built in Google Colab, designed to extract text from PDFs, summarize it, generate paraphrased versions, and evaluate their semantic similarity — all using Transformer-based models

Features:
- PDF Text → Extraction using PyPDF2

- Text Summarization → Uses BART Large CNN

- Paraphrase Generation → Uses T5 (Vamsi/T5_Paraphrase_Paws)

- Semantic Similarity Check → Uses SentenceTransformer (all-MiniLM-L6-v2)

- GPU Acceleration → Auto-detection via PyTorch

- Progress Tracking → Implemented with tqdm

- Modular Workflow → Colab Cell Segmentation

Install Dependencies



In [ ]:
!pip install -q transformers sentencepiece datasets sentence-transformers torch torchvision torchaudio PyPDF2 tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


Import Libraries


In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer, util
from google.colab import files
import PyPDF2
from tqdm import tqdm
import torch

# Check GPU availability
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"✅ Running on: {device.upper()}")


✅ Running on: CPU


Upload and Read PDF

In [ ]:
uploaded = files.upload()   # Upload your PDF
pdf_path = list(uploaded.keys())[0]

def read_pdf(path):
    """Extracts text from PDF using PyPDF2."""
    text = ""
    with open(path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for i, page in enumerate(reader.pages):
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text.strip()

pdf_text = read_pdf(pdf_path)
print(f"✅ Extracted text length: {len(pdf_text)} characters")
print("\n🔹 Preview (first 1000 chars):\n")
print(pdf_text[:1000])


Saving SOLENOID - RITHIK KUMARAN X - C.pdf to SOLENOID - RITHIK KUMARAN X - C.pdf
✅ Extracted text length: 1766 characters

🔹 Preview (first 1000 chars):

SOLENOID
- RITHIK KUMARAN K
WHAT IS A SOLENOID?
SOLENOID IS AN COIL OF MANY CIRCULAR TURNS OF INSULATED COPPER WIRE WRAPPED IN A 
WAY THAT ALL TURNS MADE ARE CLOSER TO EACH OTHER IN THE SHAPE OF A CYLINDER

SIMILAR TO A BAR MAGNET?
When we observe an Current-Carrying Solenoid, We can note that the 
magnetic field of the solenoid is similar to the bar magnet’s magnetic field and 
in fact one end of the solenoid behaves as the north pole and another as 
south pole. 

THE KEEN OBSERVATION
We can observe a keen thing about the 
magnetic field inside the solenoid.
We note that the field line’s inside the 
solenoid are parallel to each other.
What does this parallel lines tell us?      
This lines indicates us that the magnetic 
field is same at all points inside the solenoid 
(or) we can say that magnetic field is 
uniform inside the sole

Load NLP Models

In [ ]:
# Summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if device=="cuda" else -1)

# Paraphrasing model
paraphraser = pipeline("text2text-generation", model="Vamsi/T5_Paraphrase_Paws", device=0 if device=="cuda" else -1)

# Similarity model
similarity_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2", device=device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Improved Summarization with Chunking

In [ ]:
def summarize_text(text, chunk_size=1000, max_len=150, min_len=40):
    """
    Summarizes long texts by splitting them into smaller chunks.
    Handles large PDFs efficiently.
    """
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]
    summaries = []

    print(f"📘 Total Chunks: {len(chunks)}\n")
    for i, chunk in enumerate(tqdm(chunks, desc="Summarizing")):
        summary = summarizer(chunk, max_length=max_len, min_length=min_len, do_sample=False)
        summaries.append(summary[0]['summary_text'])

    return " ".join(summaries)


Paraphrasing and Similarity

In [ ]:
def generate_paraphrases(text, num_return_sequences=3):
    """Generates multiple paraphrased versions of the given text."""
    outputs = paraphraser(text, num_return_sequences=num_return_sequences, do_sample=True)
    return [o["generated_text"] for o in outputs]


def check_similarity(text1, text2):
    """Calculates cosine similarity between two sentences."""
    embeddings = similarity_model.encode([text1, text2])
    return float(util.cos_sim(embeddings[0], embeddings[1]))


End-to-End Processing Pipeline

In [ ]:
def process_text(input_text):
    summary = summarize_text(input_text)
    paraphrases = generate_paraphrases(summary, num_return_sequences=3)
    similarities = [check_similarity(summary, p) for p in paraphrases]

    return {
        "summary": summary,
        "paraphrases": paraphrases,
        "similarities": similarities
    }


Run Pipeline

In [ ]:
result = process_text(pdf_text)

print("\n" + "="*80)
print("✅ FINAL SUMMARY:\n")
print(result["summary"])

print("\n" + "="*80)
print("✅ PARAPHRASES WITH SIMILARITY SCORES:\n")
for i, (p, s) in enumerate(zip(result["paraphrases"], result["similarities"]), 1):
    print(f"{i}. {p}\n   🔹 Similarity: {s:.2f}\n")


📘 Total Chunks: 2



Summarizing: 100%|██████████| 2/2 [00:46<00:00, 23.18s/it]



✅ FINAL SUMMARY:

Solenoids are extremely useful and incredibly versatile. One end of the solenoid behaves as the north pole and another as the south pole. The magnetic field is same at all points inside thesolenoid. The field line’s inside the Solenoid are parallel to each other. The magnetic field of solenoids is the force that moves the piston and this force receives the electric current when we ring the doorbell. The strong magnetic field inside the solenoid is so strong that it can be used to magnetize a piece of soft iron.

✅ PARAPHRASES WITH SIMILARITY SCORES:

1. Solenoids are extremely useful and extremely versatile . One end of the solenoid behaves as the north pole and another as the south pole . The magnetic field of the solenoids is the force that moves the piston and this force receives the electric current when we ring the doorbell . The strong magnetic field inside the solenoid is so strong that it can be used to magnetize a piece of soft iron .
   🔹 Similarity: 0.94



Compare Custom Input

In [ ]:
custom_text = """
Artificial Intelligence (AI) is transforming industries worldwide.
From healthcare to finance, AI technologies like natural language processing and computer vision
are improving efficiency, reducing costs, and enabling new innovations.
However, challenges such as data privacy, algorithmic bias, and ethical concerns remain.
"""

result = process_text(custom_text)

print("\n🔹 Custom Summary:\n", result["summary"])
print("\n🔹 Paraphrases:")
for p, s in zip(result["paraphrases"], result["similarities"]):
    print(f"- {p} (Similarity: {s:.2f})")


📘 Total Chunks: 1



Summarizing: 100%|██████████| 1/1 [00:14<00:00, 14.66s/it]



🔹 Custom Summary:
 Artificial Intelligence (AI) is transforming industries worldwide. From healthcare to finance, AI technologies like natural language processing and computer vision are improving efficiency, reducing costs, and enabling new innovations. However, challenges such as data privacy, algorithmic bias, and ethical concerns remain.

🔹 Paraphrases:
- Artificial Intelligence (AI) is transforming industries worldwide: from healthcare to finance, AI technologies like natural language processing and computer vision are improving efficiency, reducing costs and enabling new innovations , but challenges such as data privacy, algorithmic bias and ethical concerns remain . (Similarity: 0.99)
- Artificial Intelligence (AI) is transforming industries worldwide - from healthcare to finance - AI technologies like natural language processing and computer vision are improving efficiency, reducing costs and enabling new innovations , but remain challenges such as data privacy, algorithmic bi